In [ ]:
import os
import numpy as np
import mne
from scipy.interpolate import CloughTocher2DInterpolator
from scipy.fft import fft
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Input, GlobalAveragePooling2D, Dense, Dropout
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, cohen_kappa_score

# Constants
FS = 250  # Sampling frequency (Hz)
N_FFT = 256  # FFT length
WIN_SIZE = int(0.3 * FS)  # 0.3s window (75 samples)
N_WINDOWS = 10  # Number of time windows
BANDS = [(8, 13), (13, 21), (21, 30)]  # Frequency bands (Hz)
IMG_SIZE = 64  # Image dimensions
# Standard electrode positions (Cz at origin)
ELECTRODE_POS = {'C3': (-1, 0), 'Cz': (0, 0), 'C4': (1, 0)}

def load_data(subject, session):
    """Load EEG data and events for a subject with proper scaling and timing"""
    filename = f'B{subject:02d}{session}.gdf'
    filepath = os.path.join('/kaggle/input/bci-competition-2008-graz-data-set-b/BCICIV_2b_gdf', filename)
    if not os.path.exists(filepath):
        print(f"[ERROR] File not found: {filepath}")
        return np.array([]), np.array([])
    
    try:
        # Load with scaling to microvolts (uV)
        raw = mne.io.read_raw_gdf(filepath, preload=True, verbose='ERROR')
        raw.apply_function(lambda x: x * 1e6)  # Convert to microvolts
        events, event_ids = mne.events_from_annotations(raw, verbose='ERROR')
        
        print(f"\n-- Session {session} --")
        print(f"[SUCCESS] Loading file: {filepath}")
        print(f"Found {len(events)} events. Event IDs: {event_ids}")
        
        trials, labels = [], []
        for event in events:
            if event[2] in [event_ids.get('769', -1), event_ids.get('770', -1)]:
                # Start immediately at cue onset (paper uses 3s from cue start)
                start = event[0]
                end = start + int(3 * FS)  # 3 seconds of data
                
                if end > raw.n_times:
                    print(f"Event {event} out of bounds: {end} > {raw.n_times}")
                    continue
                    
                # Get only C3, Cz, C4 channels (first 3)
                trial = raw.get_data()[:3, start:end]
                
                # Log trial stats
                print(f"Event {event}: Label={'left' if event[2]==event_ids['769'] else 'right'}, "
                      f"Start={start}, End={end}, Shape={trial.shape}, "
                      f"Mean={np.mean(trial):.2f}uV, Std={np.std(trial):.2f}uV")
                
                trials.append(trial)
                labels.append(0 if event[2] == event_ids.get('769', -1) else 1)
                
        print(f"Extracted {len(trials)} trials from session")
        return np.array(trials), np.array(labels)
    
    except Exception as e:
        print(f"[ERROR] Processing {filepath}: {str(e)}")
        return np.array([]), np.array([])

def compute_band_power(data, band):
    """Compute band power using spectral energy (FFT magnitudes)"""
    band_powers = []
    for window in np.array_split(data, N_WINDOWS, axis=1):
        # Compute FFT
        spec = fft(window, n=N_FFT, axis=1)
        freqs = np.fft.fftfreq(N_FFT, 1/FS)
        
        # Find frequency indices in band
        band_idx = np.where((freqs >= band[0]) & (freqs <= band[1]))[0]
        if len(band_idx) == 0:
            print(f"No frequencies found in band {band}!")
            band_powers.append(np.zeros(data.shape[0]))
            continue
        
        # Compute power spectral density (|X(f)|^2)
        power_spectrum = np.abs(spec[:, band_idx]) ** 2
        
        # Sum power in frequency band
        power = np.sum(power_spectrum, axis=1)
        band_powers.append(power)
    
    # Average power across windows
    return np.mean(band_powers, axis=0)

def create_tpct_image(trial_data):
    """Create TPCT image with proper scaling and interpolation"""
    print(f"\nCreating TPCT image for trial")
    print(f"Trial data shape: {trial_data.shape} (channels x samples)")
    print(f"Data stats: Min={np.min(trial_data):.2f}uV, Max={np.max(trial_data):.2f}uV, "
          f"Mean={np.mean(trial_data):.2f}uV")
    
    features = []
    for band_idx, band in enumerate(BANDS):
        print(f"\nProcessing band {band_idx+1}: {band[0]}-{band[1]}Hz")
        band_power = compute_band_power(trial_data, band)
        print(f"Band power: C3={band_power[0]:.4f}, Cz={band_power[1]:.4f}, C4={band_power[2]:.4f}")
        features.append(band_power)
    
    features = np.array(features).T  # Shape: (3 electrodes, 3 bands)
    print(f"\nFeature matrix:\n{features}")

    # Apply logarithmic scaling to handle microvolts
    features = np.log10(features + 1e-12)  # Add small constant to avoid log(0)
    print(f"Log-scaled features:\n{features}")

    # Create positions for interpolation
    positions, values = [], []
    for i, (electrode, pos) in enumerate(ELECTRODE_POS.items()):
        # Single position per electrode (no artificial shifts)
        positions.append(pos)
        values.append(features[i, 0])  # Band 1
        positions.append(pos)
        values.append(features[i, 1])  # Band 2
        positions.append(pos)
        values.append(features[i, 2])  # Band 3

    # Create interpolation grid
    x_grid = np.linspace(-1.2, 1.2, IMG_SIZE)
    y_grid = np.linspace(-1.2, 1.2, IMG_SIZE)
    xx, yy = np.meshgrid(x_grid, y_grid)
    grid_points = np.column_stack([xx.ravel(), yy.ravel()])
    
    # Interpolate each band separately
    image = np.zeros((IMG_SIZE, IMG_SIZE, 3))
    for band_idx in range(3):
        band_values = [values[i] for i in range(band_idx, len(values), 3)]
        try:
            interpolator = CloughTocher2DInterpolator(
                np.array(positions)[0::3],  # Original positions only
                band_values
            )
            band_image = interpolator(grid_points).reshape(IMG_SIZE, IMG_SIZE)
            band_image = np.nan_to_num(band_image, nan=0)
            
            # Normalize band
            band_min, band_max = band_image.min(), band_image.max()
            if band_max - band_min > 1e-8:
                band_image = (band_image - band_min) / (band_max - band_min)
                
            image[..., band_idx] = band_image
            print(f"Band {band_idx+1} image: Min={band_min:.4f}, Max={band_max:.4f}")
        except Exception as e:
            print(f"Interpolation error for band {band_idx}: {str(e)}")
            image[..., band_idx] = np.zeros((IMG_SIZE, IMG_SIZE))
    
    print(f"Image shape: {image.shape}, Range: {np.min(image):.4f}-{np.max(image):.4f}")
    return image

def build_mvgg(input_shape=(64, 64, 3), num_classes=2):
    """Build modified VGG network per paper specifications"""
    inputs = Input(shape=input_shape)
    
    # Simplified architecture based on paper
    x = Conv2D(32, (5, 5), activation='relu', padding='same')(inputs)
    x = Conv2D(32, (5, 5), activation='relu', padding='same')(x)
    x = Conv2D(64, (3, 3), strides=2, activation='relu', padding='same')(x)
    
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(128, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    x = Conv2D(128, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(128, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(256, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def main():
    all_images, all_labels = [], []
    
    print("Processing all subjects and sessions...")
    for subject in range(1, 10):
        print(f"\n{'='*50}")
        print(f"Processing Subject {subject}")
        print(f"{'='*50}")
        
        sessions = ['01T', '02T', '03T', '04E', '05E']
        for session in sessions:
            trials, labels = load_data(subject, session)
            if len(trials) == 0:
                print(f"Skipping subject {subject} session {session}")
                continue
                
            print(f"\nProcessing {len(trials)} trials for subject {subject} session {session}...")
            for i, trial in enumerate(trials):
                print(f"\nTrial {i+1}/{len(trials)}")
                image = create_tpct_image(trial)
                all_images.append(image)
            all_labels.extend(labels)
    
    X = np.array(all_images)
    y = np.array(all_labels)
    print(f"\nTotal dataset size: {X.shape[0]} samples")
    
    if len(X) == 0:
        print("No data found - exiting")
        return
    
    # 5-fold cross-validation (per paper)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies, kappas = [], []
    
    for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
        print(f"\n{'='*50}")
        print(f"Training fold {fold+1}/5")
        print(f"{'='*50}")
        
        model = build_mvgg()
        history = model.fit(X[train_idx], y[train_idx], 
                            epochs=30,
                            batch_size=32,
                            validation_split=0.2,
                            verbose=1)
        
        y_pred = model.predict(X[test_idx]).argmax(axis=1)
        acc = accuracy_score(y[test_idx], y_pred)
        kappa = cohen_kappa_score(y[test_idx], y_pred)
        
        accuracies.append(acc)
        kappas.append(kappa)
        print(f'Fold {fold+1} Accuracy: {acc:.4f}, Kappa: {kappa:.4f}')
    
    print(f'\nFinal Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}')
    print(f'Final Kappa: {np.mean(kappas):.4f} ± {np.std(kappas):.4f}')

if __name__ == "__main__":
    main()

In [8]:
import os
import numpy as np
import mne
from scipy.interpolate import CloughTocher2DInterpolator, LinearNDInterpolator, NearestNDInterpolator
from scipy.fft import fft, ifft
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, cohen_kappa_score

# ======================== CONSTANTS ========================
FS = 250  # Sampling frequency (Hz)
N_FFT = 256  # FFT length
N_WINDOWS = 10  # Number of time windows
BANDS = [(8, 13), (13, 21), (21, 30)]  # Frequency bands (Hz)
IMG_SIZE = 64  # Image dimensions
DELTA = 0.05  # Increased shift value for better separation

ELECTRODE_POS = {
    'C3': [
        (-1 - DELTA, -DELTA),  # Band 1: left and down
        (-1, 0),               # Band 2: center
        (-1 + DELTA, DELTA)    # Band 3: right and up
    ],
    'Cz': [
        (0 - DELTA, DELTA),    # Band 1: left and up
        (0, 0),                # Band 2: center
        (0 + DELTA, -DELTA)    # Band 3: right and down
    ],
    'C4': [
        (1 - DELTA, -DELTA),   # Band 1: left and down
        (1, 0),                # Band 2: center
        (1 + DELTA, DELTA)     # Band 3: right and up
    ]
}

# ======================== DATA LOADING ========================
def load_data(subject, session):
    """Load EEG data and events with proper scaling and timing"""
    filename = f'B{subject:02d}{session}.gdf'
    filepath = os.path.join('/kaggle/input/bci-competition-2008-graz-data-set-b/BCICIV_2b_gdf', filename)
    
    if not os.path.exists(filepath):
        print(f"[ERROR] File not found: {filepath}")
        return np.array([]), np.array([])
    
    try:
        print(f"\n=== Loading subject {subject}, session {session} ===")
        
        # Load and scale to microvolts
        raw = mne.io.read_raw_gdf(filepath, preload=True, verbose='ERROR')
        raw.apply_function(lambda x: x * 1e6)
        events, event_ids = mne.events_from_annotations(raw, verbose='ERROR')
        
        print(f"Found {len(events)} events")
        
        trials, labels = [], []
        valid_events = 0
        
        for event in events:
            if event[2] in [event_ids.get('769', -1), event_ids.get('770', -1)]:
                start = event[0]
                end = start + int(3 * FS)  # 3 seconds of data
                
                if end > raw.n_times:
                    print(f"  - Event at {start} exceeds data length ({raw.n_times}), skipping")
                    continue
                    
                # Get only C3, Cz, C4 channels
                trial = raw.get_data()[:3, start:end]
                trials.append(trial)
                
                label = 0 if event[2] == event_ids.get('769', -1) else 1
                labels.append(label)
                valid_events += 1
                
                # Print detailed info for first event
                if valid_events == 1:
                    print(f"\nFirst trial details:")
                    print(f"  Shape: {trial.shape} (channels x samples)")
                    print(f"  Min: {np.min(trial):.2f}μV, Max: {np.max(trial):.2f}μV")
                    print(f"  Mean: {np.mean(trial):.2f}μV, Std: {np.std(trial):.2f}μV")
                    print(f"  Variance: {np.var(trial):.2f}μV²")
                    print(f"  Channel means: C3={np.mean(trial[0]):.2f}μV, Cz={np.mean(trial[1]):.2f}μV, C4={np.mean(trial[2]):.2f}μV")
                
        print(f"\nExtracted {valid_events} valid trials")
        print(f"Label distribution: Left={labels.count(0)}, Right={labels.count(1)}")
        return np.array(trials), np.array(labels)
    
    except Exception as e:
        print(f"[ERROR] Processing file: {str(e)}")
        return np.array([]), np.array([])

# ======================== TPCT FEATURE EXTRACTION ========================
def compute_band_features(trial_data, band, verbose=False):
    """Compute band features using paper's method (FFT → sub-band → IFFT → power)"""
    window_size = trial_data.shape[1] // N_WINDOWS
    band_powers = []
    
    if verbose:
        print(f"\nComputing features for band {band[0]}-{band[1]}Hz")
        print(f"  Trial shape: {trial_data.shape}")
        print(f"  Window size: {window_size} samples")
    
    for win_idx in range(N_WINDOWS):
        start = win_idx * window_size
        end = start + window_size
        window = trial_data[:, start:end]
        
        # FFT with zero-padding
        spec = fft(window, n=N_FFT, axis=1)
        freqs = np.fft.fftfreq(N_FFT, 1/FS)
        
        # Extract sub-band frequencies
        band_idx = np.where((freqs >= band[0]) & (freqs <= band[1]))[0]
        if len(band_idx) == 0:
            band_powers.append(np.zeros(trial_data.shape[0]))
            continue
            
        sub_band_spec = spec[:, band_idx]
        
        # IFFT → Time-domain signal
        time_domain = ifft(sub_band_spec, axis=1)
        
        # Compute power (Eq 10)
        power = np.mean(np.abs(time_domain)**2, axis=1)
        band_powers.append(power)
        
        if verbose and win_idx == 0:
            print(f"  Window 1 power: C3={power[0]:.2f}, Cz={power[1]:.2f}, C4={power[2]:.2f}")
    
    # Average power across windows (Eq 11)
    avg_power = np.mean(band_powers, axis=0)
    
    if verbose:
        print(f"\nBand {band[0]}-{band[1]}Hz average power:")
        print(f"  C3: {avg_power[0]:.2f} (Min: {np.min([p[0] for p in band_powers]):.2f}, Max: {np.max([p[0] for p in band_powers]):.2f})")
        print(f"  Cz: {avg_power[1]:.2f} (Min: {np.min([p[1] for p in band_powers]):.2f}, Max: {np.max([p[1] for p in band_powers]):.2f})")
        print(f"  C4: {avg_power[2]:.2f} (Min: {np.min([p[2] for p in band_powers]):.2f}, Max: {np.max([p[2] for p in band_powers]):.2f})")
    
    return avg_power

def create_tpct_image(trial_data, verbose=False):
    """Create TPCT image with robust interpolation"""
    if verbose:
        print("\n" + "="*60)
        print("Creating TPCT Image")
        print("="*60)
        print(f"Input trial shape: {trial_data.shape}")
        print(f"Min: {np.min(trial_data):.2f}μV, Max: {np.max(trial_data):.2f}μV")
        print(f"Mean: {np.mean(trial_data):.2f}μV, Std: {np.std(trial_data):.2f}μV")
        print(f"Variance: {np.var(trial_data):.2f}μV²")
    
    features = []
    for band in BANDS:
        band_features = compute_band_features(trial_data, band, verbose=verbose)
        features.append(band_features)
    
    features = np.array(features).T  # Shape: (3 electrodes, 3 bands)
    
    if verbose:
        print("\nRaw features (electrode x band):")
        print(features)
    
    # Apply logarithmic scaling
    features = np.log10(features + 1e-12)
    
    if verbose:
        print("\nLog-scaled features:")
        print(features)
        print(f"Feature stats - Min: {np.min(features):.4f}, Max: {np.max(features):.4f}")
        print(f"Mean: {np.mean(features):.4f}, Std: {np.std(features):.4f}")
        print(f"Variance: {np.var(features):.4f}")

    positions, values = [], []
    for elec_idx, electrode in enumerate(['C3', 'Cz', 'C4']):
        for band_idx in range(3):
            pos = ELECTRODE_POS[electrode][band_idx]
            positions.append(pos)
            values.append(features[elec_idx, band_idx])
    
    if verbose:
        print("\nElectrode positions and values:")
        for i, (pos, val) in enumerate(zip(positions, values)):
            elec = ['C3', 'Cz', 'C4'][i // 3]
            band = ['μ (8-13Hz)', 'β1 (13-21Hz)', 'β2 (21-30Hz)'][i % 3]
            print(f"  {elec} {band}: Position={pos}, Value={val:.4f}")
    
    # Create grid with expanded boundaries
    x_grid = np.linspace(-1.5, 1.5, IMG_SIZE)
    y_grid = np.linspace(-0.5, 0.5, IMG_SIZE)
    xx, yy = np.meshgrid(x_grid, y_grid)
    grid_points = np.column_stack([xx.ravel(), yy.ravel()])
    
    image = np.zeros((IMG_SIZE, IMG_SIZE, 3))
    
    for band_idx in range(3):
        band_values = values[band_idx::3]
        band_positions = positions[band_idx::3]
        
        # FIX: Increased jitter magnitude to prevent collinear points
        jitter = np.random.normal(0, 0.001, (len(band_positions), 2))  # Increased from 1e-5 to 0.001
        band_positions_jittered = np.array(band_positions) + jitter
        
        try:
            interpolator = CloughTocher2DInterpolator(band_positions_jittered, band_values)
            band_image = interpolator(grid_points).reshape(IMG_SIZE, IMG_SIZE)
            method = "Clough-Tocher"
        except:
            try:
                # FIX: First fallback to linear interpolation
                interpolator = LinearNDInterpolator(band_positions_jittered, band_values)
                band_image = interpolator(grid_points).reshape(IMG_SIZE, IMG_SIZE)
                method = "Linear"
            except:
                # FIX: Final fallback to nearest neighbor interpolation
                interpolator = NearestNDInterpolator(band_positions_jittered, band_values)
                band_image = interpolator(grid_points).reshape(IMG_SIZE, IMG_SIZE)
                method = "Nearest"
        
        band_image = np.nan_to_num(band_image, nan=0)
        
        # Normalize band
        band_min, band_max = band_image.min(), band_image.max()
        if band_max - band_min > 1e-8:
            band_image = (band_image - band_min) / (band_max - band_min)
        else:
            # Handle constant band case
            band_image = np.zeros_like(band_image)
            
        image[..., band_idx] = band_image
        
        if verbose:
            band_name = ['μ (8-13Hz)', 'β1 (13-21Hz)', 'β2 (21-30Hz)'][band_idx]
            print(f"\n{band_name} band image ({method}):")
            print(f"  Raw: Min={np.min(band_image):.4f}, Max={np.max(band_image):.4f}, Mean={np.mean(band_image):.4f}")
            print(f"  Normalized: Min={np.min(image[..., band_idx]):.4f}, Max={np.max(image[..., band_idx]):.4f}")
            print(f"  Jitter magnitude: {np.max(np.abs(jitter)):.4f}")
    
    if verbose:
        print("\nFinal TPCT image:")
        print(f"  Shape: {image.shape}")
        print(f"  Min: {np.min(image):.4f}, Max: {np.max(image):.4f}")
        print(f"  Mean: {np.mean(image):.4f}, Std: {np.std(image):.4f}")
        print(f"  Variance: {np.var(image):.6f}")
        print("="*60)
    
    return image

# ======================== PAPER'S MVGG ARCHITECTURE ========================
def build_mvgg(input_shape=(64, 64, 3)):
    """Build exact mVGG architecture from paper (Table I)"""
    print("\nBuilding mVGG model architecture")
    
    inputs = Input(shape=input_shape)
    x = inputs
    
    # Section 1: 6x [Conv3x3-64]
    for i in range(6):
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(64, (2, 2), strides=2, activation='relu', padding='same')(x)  # Downsample
    
    # Section 2: 5x [Conv2x2-128]
    for i in range(5):
        x = Conv2D(128, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(128, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    # Section 3: 5x [Conv2x2-256]
    for i in range(5):
        x = Conv2D(256, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(256, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    # Section 4: 5x [Conv2x2-512]
    for i in range(5):
        x = Conv2D(512, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(512, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    # Section 5: 5x [Conv2x2-512]
    for i in range(5):
        x = Conv2D(512, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(512, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    # Classification head
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(2, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("Model summary:")
    model.summary()
    print(f"Total parameters: {model.count_params()}")
    return model

# ======================== MAIN EXECUTION ========================
def main():
    # Load all data
    all_images, all_labels = [], []
    total_trials = 0
    
    print("\n" + "="*80)
    print("STARTING DATA PROCESSING")
    print("="*80)
    
    for subject in range(1, 10):  # Subjects 1-9
        sessions = ['01T', '02T', '03T']
        print(f"\nProcessing subject {subject}/9")
        
        subject_trials = 0
        for session_idx, session in enumerate(sessions):
            print(f"\n  Session {session_idx+1}/3: {session}")
            
            trials, labels = load_data(subject, session)
            if len(trials) == 0:
                print("  -> Skipped (no trials)")
                continue
                
            print(f"  -> Processing {len(trials)} trials")
            
            for trial_idx, trial in enumerate(trials):
                # Only show details for first trial of first session
                verbose = (subject == 1 and session == '01T' and trial_idx == 0)
                
                if verbose:
                    print(f"\n    Processing trial {trial_idx+1}/{len(trials)} (VERBOSE OUTPUT)")
                    image = create_tpct_image(trial, verbose=True)
                else:
                    if trial_idx == 0:
                        print(f"    Processing {len(trials)} trials...")
                    image = create_tpct_image(trial, verbose=False)
                
                all_images.append(image)
                subject_trials += 1
            
            all_labels.extend(labels)
            total_trials += len(trials)
        
        print(f"\n  Subject {subject} summary: {subject_trials} trials processed")
    
    X = np.array(all_images)
    y = np.array(all_labels)
    
    print("\n" + "="*80)
    print("DATASET SUMMARY")
    print("="*80)
    print(f"Total trials: {X.shape[0]}")
    print(f"Image shape: {X.shape[1:]} (height x width x channels)")
    print(f"Class distribution: Left={np.sum(y==0)}, Right={np.sum(y==1)}")
    print(f"Data stats - Min: {np.min(X):.4f}, Max: {np.max(X):.4f}")
    print(f"Mean: {np.mean(X):.4f}, Std: {np.std(X):.4f}")
    print(f"Variance: {np.var(X):.6f}")
    
    # 10-fold cross-validation as in paper
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    accuracies, kappas = [], []
    
    print("\n" + "="*80)
    print("STARTING 10-FOLD CROSS VALIDATION")
    print("="*80)
    
    for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
        print(f"\nFold {fold+1}/10")
        print(f"  Train samples: {len(train_idx)} ({len(train_idx)/len(X)*100:.1f}%)")
        print(f"  Test samples: {len(test_idx)} ({len(test_idx)/len(X)*100:.1f}%)")
        print(f"  Train class distribution: Left={np.sum(y[train_idx]==0)}, Right={np.sum(y[train_idx]==1)}")
        print(f"  Test class distribution: Left={np.sum(y[test_idx]==0)}, Right={np.sum(y[test_idx]==1)}")
        
        model = build_mvgg()
        early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
        
        print("\n  Training model...")
        history = model.fit(
            X[train_idx], y[train_idx],
            validation_split=0.1,
            epochs=100,
            batch_size=32,
            callbacks=[early_stop],
            verbose=1
        )
        
        print("\n  Evaluating model...")
        y_pred = model.predict(X[test_idx], verbose=0).argmax(axis=1)
        acc = accuracy_score(y[test_idx], y_pred)
        kappa = cohen_kappa_score(y[test_idx], y_pred)
        
        accuracies.append(acc)
        kappas.append(kappa)
        print(f"\n  Fold {fold+1} results:")
        print(f"    Accuracy: {acc:.4f}")
        print(f"    Kappa:    {kappa:.4f}")
        print(f"    Error:    {1-acc:.4f}")
    
    print("\n" + "="*80)
    print("FINAL RESULTS")
    print("="*80)
    print(f"Mean Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
    print(f"Mean Kappa:    {np.mean(kappas):.4f} ± {np.std(kappas):.4f}")
    print(f"Min Accuracy:  {np.min(accuracies):.4f}, Max Accuracy: {np.max(accuracies):.4f}")
    print(f"Min Kappa:     {np.min(kappas):.4f}, Max Kappa:    {np.max(kappas):.4f}")

if __name__ == "__main__":
    # Configure GPU memory growth
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    
    # Set random seeds for reproducibility
    np.random.seed(42)
    tf.random.set_seed(42)
    
    main()


STARTING DATA PROCESSING

Processing subject 1/9

  Session 1/3: 01T

=== Loading subject 1, session 01T ===
Found 271 events

First trial details:
  Shape: (3, 750) (channels x samples)
  Min: -9.88μV, Max: 10.97μV
  Mean: -0.01μV, Std: 3.02μV
  Variance: 9.10μV²
  Channel means: C3=0.09μV, Cz=-0.16μV, C4=0.05μV

Extracted 120 valid trials
Label distribution: Left=60, Right=60
  -> Processing 120 trials

    Processing trial 1/120 (VERBOSE OUTPUT)

Creating TPCT Image
Input trial shape: (3, 750)
Min: -9.88μV, Max: 10.97μV
Mean: -0.01μV, Std: 3.02μV
Variance: 9.10μV²

Computing features for band 8-13Hz
  Trial shape: (3, 750)
  Window size: 75 samples
  Window 1 power: C3=892.15, Cz=566.63, C4=1059.41

Band 8-13Hz average power:
  C3: 1035.13 (Min: 26.89, Max: 4275.71)
  Cz: 948.81 (Min: 55.81, Max: 3476.82)
  C4: 528.74 (Min: 78.32, Max: 1059.41)

Computing features for band 13-21Hz
  Trial shape: (3, 750)
  Window size: 75 samples
  Window 1 power: C3=179.98, Cz=65.21, C4=138.37

Ba

KeyboardInterrupt: 

In [16]:
import os
import numpy as np
import mne
from scipy.interpolate import CloughTocher2DInterpolator, NearestNDInterpolator, RBFInterpolator
from scipy.fft import fft, ifft
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, cohen_kappa_score

# ======================== CONSTANTS ========================
FS = 250  # Sampling frequency (Hz)
N_FFT = 256  # FFT length
N_WINDOWS = 10  # Number of time windows
BANDS = [(8, 13), (13, 21), (21, 30)]  # Frequency bands (Hz)
IMG_SIZE = 64  # Image dimensions
DELTA = 0.05  # Shift value for electrode positioning

ELECTRODE_POS = {
    'C3': [
        (-1 - DELTA, -DELTA),  # Band 1: left and down
        (-1, 0),               # Band 2: center
        (-1 + DELTA, DELTA)    # Band 3: right and up
    ],
    'Cz': [
        (0 - DELTA, DELTA),    # Band 1: left and up
        (0, 0),                # Band 2: center
        (0 + DELTA, -DELTA)    # Band 3: right and down
    ],
    'C4': [
        (1 - DELTA, -DELTA),   # Band 1: left and down
        (1, 0),                # Band 2: center
        (1 + DELTA, DELTA)     # Band 3: right and up
    ]
}

# ======================== DATA LOADING ========================
def load_data(subject, session):
    """Load EEG data and events with proper scaling and timing"""
    filename = f'B{subject:02d}{session}.gdf'
    filepath = os.path.join('/kaggle/input/bci-competition-2008-graz-data-set-b/BCICIV_2b_gdf', filename)
    
    if not os.path.exists(filepath):
        print(f"[ERROR] File not found: {filepath}")
        return np.array([]), np.array([])
    
    try:
        # Load and scale to microvolts
        raw = mne.io.read_raw_gdf(filepath, preload=True, verbose='ERROR')
        raw.apply_function(lambda x: x * 1e6)
        events, event_ids = mne.events_from_annotations(raw, verbose='ERROR')
        
        trials, labels = [], []
        
        for event in events:
            if event[2] in [event_ids.get('769', -1), event_ids.get('770', -1)]:
                start = event[0]
                end = start + int(3 * FS)  # 3 seconds of data
                
                if end > raw.n_times:
                    continue
                    
                # Get only C3, Cz, C4 channels
                trial = raw.get_data()[:3, start:end]
                trials.append(trial)
                
                label = 0 if event[2] == event_ids.get('769', -1) else 1
                labels.append(label)
                
        return np.array(trials), np.array(labels)
    
    except Exception as e:
        print(f"[ERROR] Processing file: {str(e)}")
        return np.array([]), np.array([])

# ======================== TPCT FEATURE EXTRACTION ========================
def compute_band_features(trial_data, band):
    """Compute band features using paper's method (FFT → sub-band → IFFT → power)"""
    window_size = trial_data.shape[1] // N_WINDOWS
    band_powers = []
    
    for win_idx in range(N_WINDOWS):
        start = win_idx * window_size
        end = start + window_size
        window = trial_data[:, start:end]
        
        # FFT with zero-padding
        spec = fft(window, n=N_FFT, axis=1)
        freqs = np.fft.fftfreq(N_FFT, 1/FS)
        
        # Extract sub-band frequencies
        band_idx = np.where((freqs >= band[0]) & (freqs <= band[1]))[0]
        if len(band_idx) == 0:
            band_powers.append(np.zeros(trial_data.shape[0]))
            continue
            
        sub_band_spec = spec[:, band_idx]
        
        # IFFT → Time-domain signal
        time_domain = ifft(sub_band_spec, axis=1)
        
        # Compute power
        power = np.mean(np.abs(time_domain)**2, axis=1)
        band_powers.append(power)
    
    # Average power across windows
    return np.mean(band_powers, axis=0)

def create_tpct_image(trial_data):
    """Create TPCT image with robust interpolation"""
    features = []
    for band in BANDS:
        band_features = compute_band_features(trial_data, band)
        features.append(band_features)
    
    features = np.array(features).T  # Shape: (3 electrodes, 3 bands)
    
    # Apply logarithmic scaling
    features = np.log10(features + 1e-6)

    positions, values = [], []
    for elec_idx, electrode in enumerate(['C3', 'Cz', 'C4']):
        for band_idx in range(3):
            pos = ELECTRODE_POS[electrode][band_idx]
            positions.append(pos)
            values.append(features[elec_idx, band_idx])
    
    # Create grid
    x_grid = np.linspace(-1.5, 1.5, IMG_SIZE)
    y_grid = np.linspace(-0.5, 0.5, IMG_SIZE)
    xx, yy = np.meshgrid(x_grid, y_grid)
    grid_points = np.column_stack([xx.ravel(), yy.ravel()])
    
    image = np.zeros((IMG_SIZE, IMG_SIZE, 3))
    
    for band_idx in range(3):
        band_values = values[band_idx::3]
        band_positions = positions[band_idx::3]
        
        # Add jitter to prevent collinear points
        jitter = np.random.uniform(-0.1, 0.1, (len(band_positions), 2))
        band_positions_jittered = np.array(band_positions) + jitter
        
        # Enhanced interpolation with fallbacks
        try:
            interpolator = CloughTocher2DInterpolator(band_positions_jittered, band_values)
            band_image = interpolator(grid_points).reshape(IMG_SIZE, IMG_SIZE)
        except:
            try:
                interpolator = RBFInterpolator(band_positions_jittered, band_values)
                band_image = interpolator(grid_points).reshape(IMG_SIZE, IMG_SIZE)
            except:
                interpolator = NearestNDInterpolator(band_positions_jittered, band_values)
                band_image = interpolator(grid_points).reshape(IMG_SIZE, IMG_SIZE)
        
        # Handle NaNs
        band_image = np.nan_to_num(band_image, nan=np.mean(band_values))
        
        # Normalize band
        band_min = np.min(band_image)
        band_max = np.max(band_image)
        if band_max - band_min > 1e-4:
            band_image = (band_image - band_min) / (band_max - band_min)
        else:
            band_mean = np.mean(band_image)
            band_std = np.std(band_image)
            if band_std > 1e-6:
                band_image = 1 / (1 + np.exp(-(band_image - band_mean) / band_std))
        
        image[..., band_idx] = band_image
    
    return image

# ======================== PAPER'S MVGG ARCHITECTURE ========================
def build_mvgg(input_shape=(64, 64, 3)):
    """Build mVGG architecture from paper"""
    inputs = Input(shape=input_shape)
    x = inputs
    
    # Section 1: 6x [Conv3x3-64]
    for _ in range(6):
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(64, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    # Section 2: 5x [Conv2x2-128]
    for _ in range(5):
        x = Conv2D(128, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(128, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    # Section 3: 5x [Conv2x2-256]
    for _ in range(5):
        x = Conv2D(256, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(256, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    # Section 4: 5x [Conv2x2-512]
    for _ in range(5):
        x = Conv2D(512, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(512, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    # Section 5: 5x [Conv2x2-512]
    for _ in range(5):
        x = Conv2D(512, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(512, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    # Classification head
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(2, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# ======================== MAIN EXECUTION ========================
def main():
    # Load all data
    all_images, all_labels = [], []
    total_trials = 0
    
    print("Starting data processing for 9 subjects...")
    
    for subject in range(1, 10):
        sessions = ['01T', '02T', '03T']
        print(f"\nProcessing subject {subject}/9")
        
        subject_trials = 0
        for session in sessions:
            trials, labels = load_data(subject, session)
            if len(trials) == 0:
                continue
                
            print(f"  Session {session}: {len(trials)} trials")
            
            for trial in trials:
                image = create_tpct_image(trial)
                all_images.append(image)
                subject_trials += 1
            
            all_labels.extend(labels)
            total_trials += len(trials)
        
        print(f"  Subject {subject} completed: {subject_trials} trials")
    
    X = np.array(all_images)
    y = np.array(all_labels)
    
    print("\nDataset summary:")
    print(f"Total trials: {X.shape[0]}")
    print(f"Class distribution: Left={np.sum(y==0)}, Right={np.sum(y==1)}")
    
    # 10-fold cross-validation
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    accuracies, kappas = [], []
    
    print("\nStarting 10-fold cross-validation with mVGG model")
    
    for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
        print(f"\nFold {fold+1}/10")
        print(f"  Training samples: {len(train_idx)}, Test samples: {len(test_idx)}")
        print(f"  Class ratio: {np.sum(y[train_idx]==0)}L/{np.sum(y[train_idx]==1)}R train, "
              f"{np.sum(y[test_idx]==0)}L/{np.sum(y[test_idx]==1)}R test")
        
        model = build_mvgg()
        early_stop = EarlyStopping(monitor='val_loss', patience=5, 
                                  restore_best_weights=True, verbose=1)
        
        print("  Training model...")
        history = model.fit(
            X[train_idx], y[train_idx],
            validation_split=0.1,
            epochs=100,
            batch_size=32,
            callbacks=[early_stop],
            verbose=1
        )
        
        # Evaluate model
        y_pred = model.predict(X[test_idx], verbose=0).argmax(axis=1)
        acc = accuracy_score(y[test_idx], y_pred)
        kappa = cohen_kappa_score(y[test_idx], y_pred)
        
        accuracies.append(acc)
        kappas.append(kappa)
        print(f"  Fold results - Accuracy: {acc:.4f}, Kappa: {kappa:.4f}")
    
    print("\nFinal results:")
    print(f"Mean Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
    print(f"Mean Kappa:    {np.mean(kappas):.4f} ± {np.std(kappas):.4f}")

if __name__ == "__main__":
    # Configure GPU
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    
    # Set random seeds
    np.random.seed(42)
    tf.random.set_seed(42)
    
    main()

QhullError: QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull d Qt Qc Q12 Qbb Qz
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 104642130  delaunay  Qtriangulate  Qcoplanar-keep  Q12-allow-wide
  Qbbound-last  Qz-infinity-point  _pre-merge  _zero-centrum  Qinterior-keep
  Pgood  _max-width  2  Error-roundoff 1.4e-15  _one-merge 9.7e-15
  Visible-distance 2.8e-15  U-max-coplanar 2.8e-15  Width-outside 5.5e-15
  _wide-facet 1.7e-14  _maxoutside 1.1e-14

The input to qhull appears to be less than 3 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p3(v4):     0     0     1
- p1(v3):     0     0     0
- p2(v2):     1     0  0.91
- p0(v1):    -1     0  0.91

The center point is coplanar with a facet, or a vertex is coplanar
with a neighboring facet.  The maximum round off error for
computing distances is 1.4e-15.  The center point, facets and 

In [18]:
import os
import numpy as np
import mne
from scipy.interpolate import CloughTocher2DInterpolator
from scipy.fft import fft, ifft
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, cohen_kappa_score
import matplotlib.pyplot as plt

# ======================== PAPER-CORRECTED CONSTANTS ========================
FS = 250
N_FFT = 256
N_WINDOWS = 10
BANDS = [(8, 13), (13, 21), (21, 30)]  # μ, lower β, upper β
IMG_SIZE = 64
DELTA = 0.05  # Position shift
VERTICAL_OFFSET = 0.01  # Small vertical offset to break collinearity

# Corrected positions with vertical offset for band 1
ELECTRODE_POS = {
    'C3': [
        (-1, VERTICAL_OFFSET),      # Band 1 (center) - shifted UP
        (-1 - DELTA, -DELTA),      # Band 2
        (-1 + DELTA, DELTA)        # Band 3
    ],
    'Cz': [
        (0, 0),                    # Band 1 (center) - unchanged
        (0 - DELTA, DELTA),        # Band 2
        (0 + DELTA, -DELTA)        # Band 3
    ],
    'C4': [
        (1, -VERTICAL_OFFSET),     # Band 1 (center) - shifted DOWN
        (1 - DELTA, -DELTA),       # Band 2
        (1 + DELTA, DELTA)         # Band 3
    ]
}

# ======================== DATA LOADING ========================
def load_data(subject, session):
    filename = f'B{subject:02d}{session}.gdf'
    filepath = os.path.join('/kaggle/input/bci-competition-2008-graz-data-set-b/BCICIV_2b_gdf', filename)
    
    try:
        raw = mne.io.read_raw_gdf(filepath, preload=True, verbose='ERROR')
        raw.apply_function(lambda x: x * 1e6)  # μV conversion
        events, event_ids = mne.events_from_annotations(raw, verbose='ERROR')
        
        trials, labels = [], []
        for event in events:
            if event[2] in [event_ids.get('769', -1), event_ids.get('770', -1)]:
                start = event[0]
                end = start + int(3 * FS)  # 3s trial
                if end > raw.n_times: continue
                trials.append(raw.get_data()[:3, start:end])  # C3, Cz, C4
                labels.append(0 if event[2] == event_ids.get('769', -1) else 1)
                
        return np.array(trials), np.array(labels)
    except Exception as e:
        print(f"Error loading {filepath}: {str(e)}")
        return np.array([]), np.array([])

# ======================== PAPER-ACCURATE TPCT ========================
def compute_band_features(trial_data, band):
    """Paper-corrected feature calculation (Eq. 2-11)"""
    n_samples = trial_data.shape[1]
    window_size = n_samples // N_WINDOWS
    band_powers = np.zeros((N_WINDOWS, trial_data.shape[0]))
    
    for win_idx in range(N_WINDOWS):
        start = win_idx * window_size
        end = start + window_size
        window = trial_data[:, start:end]
        
        # FFT with zero-padding
        spec = fft(window, n=N_FFT, axis=1)
        freqs = np.fft.fftfreq(N_FFT, 1/FS)
        
        # Extract sub-band
        band_mask = (freqs >= band[0]) & (freqs <= band[1])
        sub_band_spec = spec[:, band_mask]
        
        # IFFT → Time-domain (Eq. 8)
        time_domain = np.real(ifft(sub_band_spec, axis=1))
        
        # Average power (Eq. 10)
        band_powers[win_idx] = np.mean(np.abs(time_domain), axis=1)
    
    # Average across windows (Eq. 11)
    return np.mean(band_powers, axis=0)

def create_tpct_image(trial_data):
    """Paper-accurate TPCT image generation"""
    features = []
    for band in BANDS:
        band_features = compute_band_features(trial_data, band)
        features.append(band_features)
    
    features = np.array(features).T  # (electrodes, bands)
    features = np.log10(features + 1e-12)  # Log scaling
    
    # Prepare interpolation points
    positions, values = [], []
    for elec_idx, electrode in enumerate(['C3', 'Cz', 'C4']):
        for band_idx in range(3):
            positions.append(ELECTRODE_POS[electrode][band_idx])
            values.append(features[elec_idx, band_idx])
    
    # Create grid
    x_grid = np.linspace(-1.5, 1.5, IMG_SIZE)
    y_grid = np.linspace(-0.5, 0.5, IMG_SIZE)
    xx, yy = np.meshgrid(x_grid, y_grid)
    grid_points = np.column_stack([xx.ravel(), yy.ravel()])
    
    # Paper-corrected interpolation
    image = np.zeros((IMG_SIZE, IMG_SIZE, 3))
    for ch in range(3):
        ch_values = values[ch::3]  # Values for this channel
        ch_positions = positions[ch::3]  # Positions for this channel
        
        interpolator = CloughTocher2DInterpolator(np.array(ch_positions), ch_values)
        ch_image = interpolator(grid_points).reshape(IMG_SIZE, IMG_SIZE)
        
        # Handle NaNs
        ch_image = np.nan_to_num(ch_image, nan=np.mean(ch_values))
        image[..., ch] = ch_image
    
    # Normalize per channel
    for ch in range(3):
        channel = image[..., ch]
        channel = (channel - np.min(channel)) / (np.max(channel) - np.min(channel) + 1e-8)
        image[..., ch] = channel
    
    return image

# ======================== PAPER-ACCURATE MVGG ========================
def build_mvgg(input_shape=(64, 64, 3)):
    """Paper-accurate mVGG implementation (Table I)"""
    inputs = Input(shape=input_shape)
    x = inputs
    
    # Section 1: 6x [Conv3x3-64]
    for _ in range(6):
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(64, (2, 2), strides=2, activation='relu', padding='same')(x)  # Downsample
    
    # Section 2: 5x [Conv2x2-128]
    for _ in range(5):
        x = Conv2D(128, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(128, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    # Section 3: 5x [Conv2x2-256]
    for _ in range(5):
        x = Conv2D(256, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(256, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    # Section 4: 5x [Conv2x2-512]
    for _ in range(5):
        x = Conv2D(512, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(512, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    # Section 5: 5x [Conv2x2-512]
    for _ in range(5):
        x = Conv2D(512, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(512, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    # Paper-corrected classification head (Section III-C.2.d)
    x = AveragePooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    outputs = Dense(2, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ======================== VALIDATION UTILITIES ========================
def visualize_sample(X, y, num=5):
    """Visualize samples to verify feature generation"""
    plt.figure(figsize=(15, 6))
    for i in range(num):
        plt.subplot(2, num, i+1)
        plt.imshow(X[y==0][i])
        plt.title(f"Left Hand (Sample {i})")
        plt.axis('off')
        
        plt.subplot(2, num, num+i+1)
        plt.imshow(X[y==1][i])
        plt.title(f"Right Hand (Sample {i})")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# ======================== MAIN EXECUTION ========================
def main():
    # Load data
    all_images, all_labels = [], []
    
    for subject in range(1, 10):  # 9 subjects
        for session in ['01T', '02T', '03T']:  # Training sessions
            trials, labels = load_data(subject, session)
            for trial in trials:
                all_images.append(create_tpct_image(trial))
            all_labels.extend(labels)
    
    X = np.array(all_images)
    y = np.array(all_labels)
    
    # Verify dataset
    print(f"Dataset shape: {X.shape}, Labels: {y.shape}")
    print(f"Class distribution: {np.sum(y==0)} Left, {np.sum(y==1)} Right")
    visualize_sample(X, y)  # Critical validation
    
    # 10-fold CV
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    accuracies, kappas = [], []
    
    for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
        print(f"\nFold {fold+1}/10")
        model = build_mvgg()
        early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
        # Train with validation split
        history = model.fit(
            X[train_idx], y[train_idx],
            validation_split=0.1,
            epochs=100,
            batch_size=32,
            callbacks=[early_stop],
            verbose=1
        )
        
        # Evaluate
        y_pred = model.predict(X[test_idx]).argmax(axis=1)
        acc = accuracy_score(y[test_idx], y_pred)
        kappa = cohen_kappa_score(y[test_idx], y_pred)
        
        accuracies.append(acc)
        kappas.append(kappa)
        print(f"Fold {fold+1} - Accuracy: {acc:.4f}, Kappa: {kappa:.4f}")
    
    # Final results
    print("\nFinal Results:")
    print(f"Mean Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
    print(f"Mean Kappa: {np.mean(kappas):.4f} ± {np.std(kappas):.4f}")

if __name__ == "__main__":
    # Configure GPU
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    
    # Set seeds for reproducibility
    np.random.seed(42)
    tf.random.set_seed(42)
    
    main()

QhullError: QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull d Qt Qc Q12 Qbb Qz
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 110373317  delaunay  Qtriangulate  Qcoplanar-keep  Q12-allow-wide
  Qbbound-last  Qz-infinity-point  _pre-merge  _zero-centrum  Qinterior-keep
  Pgood  _max-width  2  Error-roundoff 1.4e-15  _one-merge 9.7e-15
  Visible-distance 2.8e-15  U-max-coplanar 2.8e-15  Width-outside 5.5e-15
  _wide-facet 1.7e-14  _maxoutside 1.1e-14

The input to qhull appears to be less than 3 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p3(v4):     0     0     1
- p1(v3):     0     0     0
- p2(v2):     1 -0.01  0.91
- p0(v1):    -1  0.01  0.91

The center point is coplanar with a facet, or a vertex is coplanar
with a neighboring facet.  The maximum round off error for
computing distances is 1.4e-15.  The center point, facets and 

In [13]:
import os
import numpy as np
import mne
from scipy.interpolate import Rbf, CloughTocher2DInterpolator, LinearNDInterpolator, NearestNDInterpolator
from scipy.fft import fft, ifft
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, cohen_kappa_score

# ======================== CONSTANTS ========================
FS = 250  # Sampling frequency (Hz)
N_FFT = 256  # FFT length
N_WINDOWS = 10  # Number of time windows
BANDS = [(8, 13), (13, 21), (21, 30)]  # Frequency bands (Hz)
IMG_SIZE = 64  # Image dimensions
DELTA = 0.05  # Increased shift value for better separation

ELECTRODE_POS = {
    'C3': [
        (-1 - DELTA, -DELTA),  # Band 1: left and down
        (-1, 0),               # Band 2: center
        (-1 + DELTA, DELTA)    # Band 3: right and up
    ],
    'Cz': [
        (0 - DELTA, DELTA),    # Band 1: left and up
        (0, 0),                # Band 2: center
        (0 + DELTA, -DELTA)    # Band 3: right and down
    ],
    'C4': [
        (1 - DELTA, -DELTA),   # Band 1: left and down
        (1, 0),                # Band 2: center
        (1 + DELTA, DELTA)     # Band 3: right and up
    ]
}

# ======================== DATA LOADING ========================
def load_data(subject, session):
    """Load EEG data and events with proper scaling and timing"""
    filename = f'B{subject:02d}{session}.gdf'
    filepath = os.path.join('/kaggle/input/bci-competition-2008-graz-data-set-b/BCICIV_2b_gdf', filename)
    
    if not os.path.exists(filepath):
        print(f"[ERROR] File not found: {filepath}")
        return np.array([]), np.array([])
    
    try:
        print(f"\n=== Loading subject {subject}, session {session} ===")
        
        # Load and scale to microvolts
        raw = mne.io.read_raw_gdf(filepath, preload=True, verbose='ERROR')
        raw.apply_function(lambda x: x * 1e6)
        events, event_ids = mne.events_from_annotations(raw, verbose='ERROR')
        
        print(f"Found {len(events)} events")
        
        trials, labels = [], []
        valid_events = 0
        
        for event in events:
            if event[2] in [event_ids.get('769', -1), event_ids.get('770', -1)]:
                start = event[0]
                end = start + int(3 * FS)  # 3 seconds of data
                
                if end > raw.n_times:
                    print(f"  - Event at {start} exceeds data length ({raw.n_times}), skipping")
                    continue
                    
                # Get only C3, Cz, C4 channels
                trial = raw.get_data()[:3, start:end]
                trials.append(trial)
                
                label = 0 if event[2] == event_ids.get('769', -1) else 1
                labels.append(label)
                valid_events += 1
                
                # Print detailed info for first event
                if valid_events == 1:
                    print(f"\nFirst trial details:")
                    print(f"  Shape: {trial.shape} (channels x samples)")
                    print(f"  Min: {np.min(trial):.2f}μV, Max: {np.max(trial):.2f}μV")
                    print(f"  Mean: {np.mean(trial):.2f}μV, Std: {np.std(trial):.2f}μV")
                    print(f"  Variance: {np.var(trial):.2f}μV²")
                    print(f"  Channel means: C3={np.mean(trial[0]):.2f}μV, Cz={np.mean(trial[1]):.2f}μV, C4={np.mean(trial[2]):.2f}μV")
                
        print(f"\nExtracted {valid_events} valid trials")
        print(f"Label distribution: Left={labels.count(0)}, Right={labels.count(1)}")
        return np.array(trials), np.array(labels)
    
    except Exception as e:
        print(f"[ERROR] Processing file: {str(e)}")
        return np.array([]), np.array([])

# ======================== TPCT FEATURE EXTRACTION ========================
def compute_band_features(trial_data, band, verbose=False):
    """Compute band features using paper's method (FFT → sub-band → IFFT → power)"""
    window_size = trial_data.shape[1] // N_WINDOWS
    band_powers = []
    
    if verbose:
        print(f"\nComputing features for band {band[0]}-{band[1]}Hz")
        print(f"  Trial shape: {trial_data.shape}")
        print(f"  Window size: {window_size} samples")
    
    for win_idx in range(N_WINDOWS):
        start = win_idx * window_size
        end = start + window_size
        window = trial_data[:, start:end]
        
        # FFT with zero-padding
        spec = fft(window, n=N_FFT, axis=1)
        freqs = np.fft.fftfreq(N_FFT, 1/FS)
        
        # Extract sub-band frequencies
        band_idx = np.where((freqs >= band[0]) & (freqs <= band[1]))[0]
        if len(band_idx) == 0:
            band_powers.append(np.zeros(trial_data.shape[0]))
            continue
            
        sub_band_spec = spec[:, band_idx]
        
        # IFFT → Time-domain signal
        time_domain = ifft(sub_band_spec, axis=1)
        
        # Compute power (Eq 10)
        power = np.mean(np.abs(time_domain)**2, axis=1)
        band_powers.append(power)
        
        if verbose and win_idx == 0:
            print(f"  Window 1 power: C3={power[0]:.2f}, Cz={power[1]:.2f}, C4={power[2]:.2f}")
    
    # Average power across windows (Eq 11)
    avg_power = np.mean(band_powers, axis=0)
    
    if verbose:
        print(f"\nBand {band[0]}-{band[1]}Hz average power:")
        print(f"  C3: {avg_power[0]:.2f} (Min: {np.min([p[0] for p in band_powers]):.2f}, Max: {np.max([p[0] for p in band_powers]):.2f})")
        print(f"  Cz: {avg_power[1]:.2f} (Min: {np.min([p[1] for p in band_powers]):.2f}, Max: {np.max([p[1] for p in band_powers]):.2f})")
        print(f"  C4: {avg_power[2]:.2f} (Min: {np.min([p[2] for p in band_powers]):.2f}, Max: {np.max([p[2] for p in band_powers]):.2f})")
    
    return avg_power

def create_tpct_image(trial_data, verbose=False):
    """Create TPCT image with robust interpolation"""
    if verbose:
        print("\n" + "="*60)
        print("Creating TPCT Image")
        print("="*60)
        print(f"Input trial shape: {trial_data.shape}")
        print(f"Min: {np.min(trial_data):.2f}μV, Max: {np.max(trial_data):.2f}μV")
        print(f"Mean: {np.mean(trial_data):.2f}μV, Std: {np.std(trial_data):.2f}μV")
        print(f"Variance: {np.var(trial_data):.2f}μV²")
    
    features = []
    for band in BANDS:
        band_features = compute_band_features(trial_data, band, verbose=verbose)
        features.append(band_features)
    
    features = np.array(features).T  # Shape: (3 electrodes, 3 bands)
    
    if verbose:
        print("\nRaw features (electrode x band):")
        print(features)
    
    # Apply logarithmic scaling
    features = np.log10(features + 1e-12)
    
    if verbose:
        print("\nLog-scaled features:")
        print(features)
        print(f"Feature stats - Min: {np.min(features):.4f}, Max: {np.max(features):.4f}")
        print(f"Mean: {np.mean(features):.4f}, Std: {np.std(features):.4f}")
        print(f"Variance: {np.var(features):.4f}")

    positions, values = [], []
    for elec_idx, electrode in enumerate(['C3', 'Cz', 'C4']):
        for band_idx in range(3):
            pos = ELECTRODE_POS[electrode][band_idx]
            positions.append(pos)
            values.append(features[elec_idx, band_idx])
    
    if verbose:
        print("\nElectrode positions and values:")
        for i, (pos, val) in enumerate(zip(positions, values)):
            elec = ['C3', 'Cz', 'C4'][i // 3]
            band = ['μ (8-13Hz)', 'β1 (13-21Hz)', 'β2 (21-30Hz)'][i % 3]
            print(f"  {elec} {band}: Position={pos}, Value={val:.4f}")
    
    # Create grid with expanded boundaries
    x_grid = np.linspace(-1.5, 1.5, IMG_SIZE)
    y_grid = np.linspace(-0.5, 0.5, IMG_SIZE)
    xx, yy = np.meshgrid(x_grid, y_grid)
    grid_points = np.column_stack([xx.ravel(), yy.ravel()])
    
    image = np.zeros((IMG_SIZE, IMG_SIZE, 3))
    
    # ====== FIXED INTERPOLATION ======
    for band_idx in range(3):
        band_values = values[band_idx::3]
        band_positions = positions[band_idx::3]
        
        # Add unique y-offsets to break collinearity
        band_name = ['μ (8-13Hz)', 'β1 (13-21Hz)', 'β2 (21-30Hz)'][band_idx]
        y_offsets = {
            'β1 (13-21Hz)': [-0.01, 0.01, 0.0],  # Break collinearity for β1 band
            'default': [0, 0, 0]
        }
        offsets = y_offsets.get(band_name, y_offsets['default'])
        
        band_positions_shifted = []
        for i, (x, y) in enumerate(band_positions):
            band_positions_shifted.append((x, y + offsets[i]))
        
        # Convert to numpy arrays
        points = np.array(band_positions_shifted)
        values_arr = np.array(band_values)
        
        # Use RBF interpolation with fallback
        try:
            # Create RBF interpolator
            rbf = Rbf(points[:, 0], points[:, 1], values_arr, function='thin_plate')
            band_image = rbf(xx, yy)
            method = "RBF (thin_plate)"
        except Exception as e:
            if verbose:
                print(f"RBF failed: {str(e)}")
            # Fallback to nearest neighbor
            interpolator = NearestNDInterpolator(points, values_arr)
            band_image = interpolator(grid_points).reshape(IMG_SIZE, IMG_SIZE)
            method = "Nearest (fallback)"
        
        # Robust normalization
        band_range = np.max(band_image) - np.min(band_image)
        if band_range < 1e-8:  # Handle near-constant case
            band_image = np.zeros_like(band_image)
        else:
            band_image = (band_image - np.min(band_image)) / band_range
        
        image[..., band_idx] = band_image
        
        if verbose:
            print(f"\n{band_name} band image ({method}):")
            print(f"  Raw: Min={np.min(band_image):.4f}, Max={np.max(band_image):.4f}")
            print(f"  Normalized: Min={np.min(image[..., band_idx]):.4f}, Max={np.max(image[..., band_idx]):.4f}")
            print(f"  Offsets applied: {offsets}")
    # ====== END FIX ======
    
    if verbose:
        print("\nFinal TPCT image:")
        print(f"  Shape: {image.shape}")
        print(f"  Min: {np.min(image):.4f}, Max: {np.max(image):.4f}")
        print(f"  Mean: {np.mean(image):.4f}, Std: {np.std(image):.4f}")
        print(f"  Variance: {np.var(image):.6f}")
        print("="*60)
    
    return image

# ======================== PAPER'S MVGG ARCHITECTURE ========================
def build_mvgg(input_shape=(64, 64, 3)):
    """Build exact mVGG architecture from paper (Table I)"""
    print("\nBuilding mVGG model architecture")
    
    inputs = Input(shape=input_shape)
    x = inputs
    
    # Section 1: 6x [Conv3x3-64]
    for i in range(6):
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(64, (2, 2), strides=2, activation='relu', padding='same')(x)  # Downsample
    
    # Section 2: 5x [Conv2x2-128]
    for i in range(5):
        x = Conv2D(128, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(128, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    # Section 3: 5x [Conv2x2-256]
    for i in range(5):
        x = Conv2D(256, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(256, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    # Section 4: 5x [Conv2x2-512]
    for i in range(5):
        x = Conv2D(512, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(512, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    # Section 5: 5x [Conv2x2-512]
    for i in range(5):
        x = Conv2D(512, (2, 2), activation='relu', padding='same')(x)
    x = Conv2D(512, (2, 2), strides=2, activation='relu', padding='same')(x)
    
    # Classification head
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(2, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("Model summary:")
    model.summary()
    print(f"Total parameters: {model.count_params()}")
    return model

# ======================== MAIN EXECUTION ========================
def main():
    # Load all data
    all_images, all_labels = [], []
    total_trials = 0
    
    print("\n" + "="*80)
    print("STARTING DATA PROCESSING")
    print("="*80)
    
    for subject in range(1, 10):  # Subjects 1-9
        sessions = ['01T', '02T', '03T']
        print(f"\nProcessing subject {subject}/9")
        
        subject_trials = 0
        for session_idx, session in enumerate(sessions):
            print(f"\n  Session {session_idx+1}/3: {session}")
            
            trials, labels = load_data(subject, session)
            if len(trials) == 0:
                print("  -> Skipped (no trials)")
                continue
                
            print(f"  -> Processing {len(trials)} trials")
            
            for trial_idx, trial in enumerate(trials):
                # Only show details for first trial of first session
                verbose = (subject == 1 and session == '01T' and trial_idx == 0)
                
                if verbose:
                    print(f"\n    Processing trial {trial_idx+1}/{len(trials)} (VERBOSE OUTPUT)")
                    image = create_tpct_image(trial, verbose=True)
                else:
                    if trial_idx == 0:
                        print(f"    Processing {len(trials)} trials...")
                    image = create_tpct_image(trial, verbose=False)
                
                all_images.append(image)
                subject_trials += 1
            
            all_labels.extend(labels)
            total_trials += len(trials)
        
        print(f"\n  Subject {subject} summary: {subject_trials} trials processed")
    
    X = np.array(all_images)
    y = np.array(all_labels)
    
    print("\n" + "="*80)
    print("DATASET SUMMARY")
    print("="*80)
    print(f"Total trials: {X.shape[0]}")
    print(f"Image shape: {X.shape[1:]} (height x width x channels)")
    print(f"Class distribution: Left={np.sum(y==0)}, Right={np.sum(y==1)}")
    print(f"Data stats - Min: {np.min(X):.4f}, Max: {np.max(X):.4f}")
    print(f"Mean: {np.mean(X):.4f}, Std: {np.std(X):.4f}")
    print(f"Variance: {np.var(X):.6f}")
    
    # 10-fold cross-validation as in paper
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    accuracies, kappas = [], []
    
    print("\n" + "="*80)
    print("STARTING 10-FOLD CROSS VALIDATION")
    print("="*80)
    
    for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
        print(f"\nFold {fold+1}/10")
        print(f"  Train samples: {len(train_idx)} ({len(train_idx)/len(X)*100:.1f}%)")
        print(f"  Test samples: {len(test_idx)} ({len(test_idx)/len(X)*100:.1f}%)")
        print(f"  Train class distribution: Left={np.sum(y[train_idx]==0)}, Right={np.sum(y[train_idx]==1)}")
        print(f"  Test class distribution: Left={np.sum(y[test_idx]==0)}, Right={np.sum(y[test_idx]==1)}")
        
        model = build_mvgg()
        early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
        
        print("\n  Training model...")
        history = model.fit(
            X[train_idx], y[train_idx],
            validation_split=0.1,
            epochs=100,
            batch_size=32,
            callbacks=[early_stop],
            verbose=1
        )
        
        print("\n  Evaluating model...")
        y_pred = model.predict(X[test_idx], verbose=0).argmax(axis=1)
        acc = accuracy_score(y[test_idx], y_pred)
        kappa = cohen_kappa_score(y[test_idx], y_pred)
        
        accuracies.append(acc)
        kappas.append(kappa)
        print(f"\n  Fold {fold+1} results:")
        print(f"    Accuracy: {acc:.4f}")
        print(f"    Kappa:    {kappa:.4f}")
        print(f"    Error:    {1-acc:.4f}")
    
    print("\n" + "="*80)
    print("FINAL RESULTS")
    print("="*80)
    print(f"Mean Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
    print(f"Mean Kappa:    {np.mean(kappas):.4f} ± {np.std(kappas):.4f}")
    print(f"Min Accuracy:  {np.min(accuracies):.4f}, Max Accuracy: {np.max(accuracies):.4f}")
    print(f"Min Kappa:     {np.min(kappas):.4f}, Max Kappa:    {np.max(kappas):.4f}")

if __name__ == "__main__":
    # Configure GPU memory growth
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    
    # Set random seeds for reproducibility
    np.random.seed(42)
    tf.random.set_seed(42)
    
    main()


STARTING DATA PROCESSING

Processing subject 1/9

  Session 1/3: 01T

=== Loading subject 1, session 01T ===
Found 271 events

First trial details:
  Shape: (3, 750) (channels x samples)
  Min: -9.88μV, Max: 10.97μV
  Mean: -0.01μV, Std: 3.02μV
  Variance: 9.10μV²
  Channel means: C3=0.09μV, Cz=-0.16μV, C4=0.05μV

Extracted 120 valid trials
Label distribution: Left=60, Right=60
  -> Processing 120 trials

    Processing trial 1/120 (VERBOSE OUTPUT)

Creating TPCT Image
Input trial shape: (3, 750)
Min: -9.88μV, Max: 10.97μV
Mean: -0.01μV, Std: 3.02μV
Variance: 9.10μV²

Computing features for band 8-13Hz
  Trial shape: (3, 750)
  Window size: 75 samples
  Window 1 power: C3=892.15, Cz=566.63, C4=1059.41

Band 8-13Hz average power:
  C3: 1035.13 (Min: 26.89, Max: 4275.71)
  Cz: 948.81 (Min: 55.81, Max: 3476.82)
  C4: 528.74 (Min: 78.32, Max: 1059.41)

Computing features for band 13-21Hz
  Trial shape: (3, 750)
  Window size: 75 samples
  Window 1 power: C3=179.98, Cz=65.21, C4=138.37

Ba

KeyboardInterrupt: 